### Introduction

This notebook aims to find the impact of random nudges

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_synergistic_nudge_evolutionary as synergistic_nudge_ev
import maximum_nudges_evolutionary as ev_max_nudges
import nudge_non_causal
import get_data

### Notebook level constants

In [3]:
NUDGE_SIZE = 0.01
PATH_TO_DISTRIBUTIONS = "/home/joboti/azumi_derkjan/master_thesis/code/"

### Find the random impact of a nudge given the nudge type, the input and conditional output, and nudge size



In [6]:
def get_random_nudge_impacts(input_dists, cond_output_dists, nudge_size, nudge_type, 
                             backup_filename, parameters):
    """
    Find the maximum impact of the nudges
    
    Parameters:
    ----------
    input_dists: list of nd-arrays representing probability distributions
    cond_output_dists: list of nd-arrays 
        Representing output distributions (the last axis) conditioned on the input 
        distributions
    nudge_size: positive float
    nudge_type: string
        One of the following: {"individual", "local", "synergistic", "global"}
    filename_to_save: string, 
        Should be a valid path
    parameters: a dict,
        The parameters used to find the maximum nudge
    
    """
    impacts = []
    for input_dist, cond_output in zip(input_dists, cond_output_dists):
        print("count {}".format(len(impacts)))
        if nudge_type == "individual":
            new_dist = nudge_non_causal.nudge_individual_without_conditional(input_dist, nudge_size)
        elif nudge_type == "local":
            new_dist = nudge_non_causal.nudge_local(
                input_dist, nudged_vars=parameters["nudged_vars"], nudge_size=nudge_size,
                without_conditional=True
            )
        elif nudge_type == "synergistic":
            new_dist = np.copy(input_dist)
            for _ in range(parameters["max_number_of_mutations"]):
                nudge_non_causal.synergistic_mutate(
                    new_dist, parameters["mutation_size"]
                )
                new_nudge_size = np.sum(abs(new_dist-input_dist))
                adjustment_factor = nudge_size/new_nudge_size
                if adjustment_factor <= 1:
                    break
            
            new_dist = input_dist + (new_dist-input_dist)*adjustment_factor
        elif nudge_type == "global":
            new_dist = nudge_non_causal.nudge_global(input_dist, nudge_size, without_conditional=True)
        else:
            raise ValueError("provide a valid nudge type")

        l1_norm_to_old_distribution = np.sum(np.absolute(input_dist-new_dist))
        if abs(l1_norm_to_old_distribution-NUDGE_SIZE) > 10**(-7):
            print("WARNING size of nudge {}".format(l1_norm_to_old_distribution))

        impact = nudge_non_causal.find_nudge_impact(input_dist, new_dist, cond_output)
        print("the nudge impact {}".format(impact))
        impacts.append(impact)
        with open(backup_filename, 'w') as f:
            json.dump(impacts, f, indent=4)
        
    return impacts


IndentationError: unindent does not match any outer indentation level (<ipython-input-6-151290a454a3>, line 33)

### Generate the data and run the experiments

#### First for system distributions with limited entropy

In [ ]:
PERCENTAGE_MAX_ENTROPY_SIZE = 90
NUMBER_OF_VARS = 3
NUMBER_OF_STATES = 3
FILENAME_FORMAT_INPUT = "input_dist_{}.npy"
FILENAME_FORMAT_OUTPUT = "cond_output_dist_{}.npy"
DIST_START = 100
DIST_END = 200
NUDGE_TYPE = "synergistic"

distribution_numbers = list(range(DIST_START, DIST_END, 1))
if NUDGE_TYPE == "global" or NUDGE_TYPE =="individual": 
    parameters = None
elif NUDGE_TYPE == "local":
    pass
elif NUDGE_TYPE == "synergistic":
    parameter_folder = "synergistic_nudge_parameters/"
    parameter_file_name = "entropy_percentage{}_{}vars_{}states_dist_{}_{}.json".format(
        PERCENTAGE_MAX_ENTROPY_SIZE, NUMBER_OF_VARS, NUMBER_OF_STATES, DIST_START, DIST_END
    )
    parameter_file = parameter_folder + parameter_file_name     
    with open(parameter_file) as f:
        parameters = json.load(f)

backup_filename = ("data_experiments/" + 
    "backup_random_impacts_{}var_{}states_{}entropy_{}_nudge_dists{}-{}.json".format(
        NUMBER_OF_VARS, NUMBER_OF_STATES, PERCENTAGE_MAX_ENTROPY_SIZE, NUDGE_TYPE, DIST_START, DIST_END 
    )
)
path_to_limited_entropy_system_dists = (
    PATH_TO_DISTRIBUTIONS + "system_distributions/" 
    + "limited_entropy/"
)

input_dists = get_data.get_system_distributions_limited_entropy(
    path_to_limited_entropy_system_dists, PERCENTAGE_MAX_ENTROPY_SIZE,
    NUMBER_OF_VARS, NUMBER_OF_STATES, FILENAME_FORMAT_INPUT, 
    distribution_numbers
)
output_dists = get_data.get_system_distributions_limited_entropy(
    path_to_limited_entropy_system_dists, PERCENTAGE_MAX_ENTROPY_SIZE,
    NUMBER_OF_VARS, NUMBER_OF_STATES, FILENAME_FORMAT_OUTPUT, 
    distribution_numbers
)

impacts =  #find random nudge impacts

print(impacts)
filename_to_save_impacts =  "random_impacts_{}var_{}states_{}entropy_{}_nudge_dists{}-{}.json".format(
    NUMBER_OF_VARS, NUMBER_OF_STATES, PERCENTAGE_MAX_ENTROPY_SIZE, NUDGE_TYPE, DIST_START, DIST_END 
)
with open("data_experiments/" + filename_to_save_impacts, 'w') as f:
    json.dump(impacts, f)
